In [ ]:
!pip install optuna


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords, wordnet
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
import os
import string
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
from sklearn.manifold import TSNE
import itertools
import collections
from collections import Counter
from gensim import corpora, models
from gensim.models import Word2Vec, word2vec, LdaModel
import optuna
from hyperopt import hp, tpe, fmin
import warnings
warnings.filterwarnings("ignore")
import gensim
from gensim.utils import simple_preprocess
from datetime import datetime
# import spacy
from nltk import pos_tag
import pickle


In [3]:
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!ls 'drive/MyDrive/5.TopicModeling/Data'

2.TubbsFire_wildfire_words.csv	LIWC_2.CampFire_Period2.csv
final_campfire.csv		LIWC_2.Mendocino_Period2.csv
final_mendocino.csv		LIWC_2.Thomas_Period2.csv
final_thomas.csv		TubbsFire_version1.csv
final_tubbs.csv


In [9]:
df = pd.read_csv('drive/MyDrive/5.TopicModeling/Data/final_tubbs.csv', encoding='latin-1')
df['tweet_created_at'] = pd.to_datetime(df["tweet_created_at"])
df['tweet_created_at'] = df['tweet_created_at'].dt.strftime('%Y-%m-%d')
df

,Unnamed: 0,tweet_id,tweet_text,tweet_created_at,tweet_source,tweet_lang,tweet_possibly_sensitive,tweet_retweet_count,tweet_reply_count,tweet_like_count,...,place_name,place_full_name,place_country,place_country_code,place_type,clean_text,tokens,token,lemm_text,lemm_textA
0,0,9.041318e+17,"#WILDFIRE brings smoke, ash, unhealthful air q...",2017-09-02,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,wildfire brings smoke ash unhealthful air qual...,"['wildfire', 'brings', 'smoke', 'ash', 'unheal...","[('brings', 'NNS'), ('smoke', 'VBD'), ('ash', ...","['brings', 'smoke', 'ash', 'unhealthful', 'air...","['brings', 'smoke', 'ash', 'air', 'quality', '..."
1,1,9.041317e+17,"#WILDFIRE brings smoke, ash, unhealthful air q...",2017-09-02,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,wildfire brings smoke ash unhealthful air qual...,"['wildfire', 'brings', 'smoke', 'ash', 'unheal...","[('brings', 'NNS'), ('smoke', 'VBD'), ('ash', ...","['brings', 'smoke', 'ash', 'unhealthful', 'air...","['brings', 'smoke', 'ash', 'air', 'quality', '..."
2,2,9.041309e+17,California on fire and Texas under water smh,2017-09-02,NaN,en,True,0.0,0.0,13.0,...,NaN,NaN,NaN,NaN,NaN,california fire texas water smh,"['california', 'texas', 'water', 'smh']","[('california', 'NN'), ('fire', 'NN'), ('texas...","['california', 'fire', 'texas', 'water', 'smh']","['california', 'fire', 'texas', 'water', 'smh']"
3,3,9.041304e+17,MVC: 6000 block of Tom Higgins Lane. Lake Shan...,2017-09-02,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,mvc 6000 block tom higgins lane lake shannon b...,"['mvc', 'block', 'tom', 'higgins', 'lane', 'la...","[('mvc', 'NN'), ('6000', 'CD'), ('block', 'NN'...","['mvc', 'block', 'tom', 'higgins', 'lane', 'la...","['block', 'tom', 'lane', 'lake', 'boat', 'laun..."
4,4,9.041302e+17,"Wildlife brings smoke, ash, unhealthful air qu...",2017-09-02,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,wildlife brings smoke ash unhealthful air qual...,"['wildlife', 'brings', 'smoke', 'ash', 'unheal...","[('wildlife', 'JJ'), ('brings', 'NNS'), ('smok...","['wildlife', 'brings', 'smoke', 'ash', 'unheal...","['wildlife', 'brings', 'smoke', 'ash', 'air', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301471,301471,9.415234e+17,"Pasadena, California â¢ \nJet Propulsion Labo...",2017-12-15,NaN,en,False,3.0,0.0,9.0,...,Mid-Wilshire,"Mid-Wilshire, Los Angeles",Verenigde Staten,US,neighborhood,pasadena california jet propulsion laboratory ...,"['pasadena', 'california', 'jet', 'propulsion'...","[('pasadena', 'NN'), ('california', 'NN'), ('j...","['pasadena', 'california', 'jet', 'propulsion'...","['california', 'jet', 'fire', 'personnel', 'as..."
301472,301472,9.419101e+17,So sad.........:(\nPlease help the family of t...,2017-12-16,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,sad please help family california firefighter ...,"['sad', 'help', 'family', 'california', 'firef...","[('sad', 'JJ'), ('please', 'NN'), ('help', 'VB...","['sad', 'please', 'help', 'family', 'californi...","['sad', 'please', 'help', 'family', 'californi..."
301473,301473,9.424684e+17,My heart goes out to all those affected by the...,2017-12-17,NaN,en,True,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,heart goes affected california wildfires sad,"['heart', 'goes', 'affected', 'california', 'w...","[('heart', 'NN'), ('goes', 'VBZ'), ('affected'...","['heart', 'go', 'affect', 'california', 'wildf...","['heart', 'go', 'affect', 'california', 'wildf..."
301474,301474,9.458634e+17,@realDonaldTrump Thank you Mr Trump for your i...,2017-12-27,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,thank mr trump incredible leadership divine sk...,"['thank', 'mr', 'trump', 'incredible', 'leader...","[('thank', 'JJ'), ('trump', 'NN'), ('incredibl...","['thank', 'trump', 'incredible', 'leadership',...","['thank', 'trump', 'incredible', 'leadership',..."


### separating data into three difference period

09/02/2017-10/07/2017: Before wildfire

10/08/2017-10/31/2017: During wildfire

11/01/2017-12/01/2017: After wildfire

In [10]:
#before wildfire
start_date = '2017-10-08'
end_date = '2017-10-31'

filtered_df = df[(df['tweet_created_at'] >= start_date) & (df['tweet_created_at'] <= end_date)]
filtered_df

,Unnamed: 0,tweet_id,tweet_text,tweet_created_at,tweet_source,tweet_lang,tweet_possibly_sensitive,tweet_retweet_count,tweet_reply_count,tweet_like_count,...,place_name,place_full_name,place_country,place_country_code,place_type,clean_text,tokens,token,lemm_text,lemm_textA
15265,15265,9.171776e+17,Santa Ana winds and fire warnings? It must be ...,2017-10-08,NaN,en,False,33.0,5.0,55.0,...,NaN,NaN,NaN,NaN,NaN,santa ana winds fire warnings must october sou...,"['santa', 'ana', 'winds', 'warnings', 'october...","[('santa', 'NN'), ('ana', 'NN'), ('winds', 'VB...","['santa', 'ana', 'wind', 'fire', 'warning', 'o...","['santa', 'ana', 'wind', 'fire', 'warning', 'o..."
15266,15266,9.171773e+17,Napa Fire Breaks Out Near Airport https://t.co...,2017-10-08,NaN,en,False,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,napa fire breaks near airport,"['napa', 'breaks', 'near', 'airport']","[('napa', 'JJ'), ('fire', 'NN'), ('breaks', 'N...","['napa', 'fire', 'break', 'airport']","['napa', 'fire', 'break', 'airport']"
15267,15267,9.171773e+17,Napa Fire Breaks Out Near Airport https://t.co...,2017-10-08,NaN,en,False,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,napa fire breaks near airport,"['napa', 'breaks', 'near', 'airport']","[('napa', 'JJ'), ('fire', 'NN'), ('breaks', 'N...","['napa', 'fire', 'break', 'airport']","['napa', 'fire', 'break', 'airport']"
15268,15268,9.171773e+17,People stand on a dock while fishing on Buntze...,2017-10-08,NaN,en,False,0.0,0.0,8.0,...,NaN,NaN,NaN,NaN,NaN,people stand dock fishing buntzen lake coloure...,"['people', 'stand', 'dock', 'fishing', 'buntze...","[('people', 'NNS'), ('stand', 'VBP'), ('dock',...","['people', 'stand', 'dock', 'fishing', 'buntze...","['people', 'stand', 'lake', 'smoke', 'hang', '..."
15269,15269,9.171767e+17,#FireMcAdoo #FireReese #NoGiantPride ânough ...,2017-10-08,NaN,en,False,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,fire mc adoo fire reese giant pride nough said...,"['mc', 'adoo', 'reese', 'giant', 'pride', 'nou...","[('nough', 'NN'), ('said', 'VBD')]","['nough', 'say']",['say']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301205,301205,9.182763e+17,.@Chevron: We are deeply saddened by the loss ...,2017-10-12,NaN,en,True,4.0,1.0,7.0,...,NaN,NaN,NaN,NaN,NaN,deeply saddened loss life devastation caused w...,"['deeply', 'saddened', 'loss', 'life', 'devast...","[('deeply', 'RB'), ('saddened', 'VBN'), ('loss...","['deeply', 'sadden', 'loss', 'life', 'devastat...","['loss', 'life', 'devastation', 'cause', 'wild..."
301206,301206,9.182728e+17,So sad. â¡ï¸ âCouple married for 75 years ...,2017-10-12,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,sad couple married 75 years die together calif...,"['sad', 'couple', 'married', 'years', 'die', '...","[('sad', 'JJ'), ('couple', 'NN'), ('married', ...","['sad', 'couple', 'marry', 'year', 'die', 'tog...","['sad', 'couple', 'marry', 'year', 'die', 'tog..."
301207,301207,9.182711e+17,3Novices:Death Toll Climbs to 21 From Wildfire...,2017-10-12,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,3novicesdeath toll climbs 21 wildfires norther...,"['novicesdeath', 'toll', 'climbs', 'wildfires'...","[('3novices', 'NNS'), ('death', 'NN'), ('toll'...","['3novices', 'death', 'toll', 'climb', 'wildfi...","['3novices', 'death', 'toll', 'climb', 'wildfi..."
301208,301208,9.196318e+17,So sad - cartoonist Brian Fies' comic about lo...,2017-10-15,NaN,en,False,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,sad cartoonist brian fies comic losing home ca...,"['sad', 'cartoonist', 'brian', 'fies', 'comic'...","[('sad', 'JJ'), ('cartoonist', 'NN'), ('brian'...","['sad', 'cartoonist', 'brian', 'fies', 'comic'...","['sad', 'brian', 'comic', 'lose', 'home', 'cal..."


# Word Distribution

In [ ]:
wordcounter = Counter()
# Update the Counter with all tokens in the 'lemm_text' column
for tokens in filtered_df['lemm_text']:
    wordcounter.update(tokens)
filtered_df['lemm_textA'] = [[word for word in tokens if wordcounter[word] >= 100] for tokens in filtered_df['lemm_text']]
filtered_df

,Unnamed: 0,tweet_id,tweet_text,tweet_created_at,tweet_source,tweet_lang,tweet_possibly_sensitive,tweet_retweet_count,tweet_reply_count,tweet_like_count,...,place_name,place_full_name,place_country,place_country_code,place_type,clean_text,tokens,token,lemm_text,lemm_textA
15265,15265,9.171776e+17,Santa Ana winds and fire warnings? It must be ...,2017-10-08,NaN,en,False,33.0,5.0,55.0,...,NaN,NaN,NaN,NaN,NaN,santa ana winds fire warnings must october sou...,"['santa', 'ana', 'winds', 'warnings', 'october...","[('santa', 'NN'), ('ana', 'NN'), ('winds', 'VB...","['santa', 'ana', 'wind', 'fire', 'warning', 'o...","[[, ', s, a, n, t, a, ', ,, , ', a, n, a, ', ..."
15266,15266,9.171773e+17,Napa Fire Breaks Out Near Airport https://t.co...,2017-10-08,NaN,en,False,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,napa fire breaks near airport,"['napa', 'breaks', 'near', 'airport']","[('napa', 'JJ'), ('fire', 'NN'), ('breaks', 'N...","['napa', 'fire', 'break', 'airport']","[[, ', n, a, p, a, ', ,, , ', f, i, r, e, ', ..."
15267,15267,9.171773e+17,Napa Fire Breaks Out Near Airport https://t.co...,2017-10-08,NaN,en,False,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,napa fire breaks near airport,"['napa', 'breaks', 'near', 'airport']","[('napa', 'JJ'), ('fire', 'NN'), ('breaks', 'N...","['napa', 'fire', 'break', 'airport']","[[, ', n, a, p, a, ', ,, , ', f, i, r, e, ', ..."
15268,15268,9.171773e+17,People stand on a dock while fishing on Buntze...,2017-10-08,NaN,en,False,0.0,0.0,8.0,...,NaN,NaN,NaN,NaN,NaN,people stand dock fishing buntzen lake coloure...,"['people', 'stand', 'dock', 'fishing', 'buntze...","[('people', 'NNS'), ('stand', 'VBP'), ('dock',...","['people', 'stand', 'dock', 'fishing', 'buntze...","[[, ', p, e, o, p, l, e, ', ,, , ', s, t, a, ..."
15269,15269,9.171767e+17,#FireMcAdoo #FireReese #NoGiantPride ânough ...,2017-10-08,NaN,en,False,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,fire mc adoo fire reese giant pride nough said...,"['mc', 'adoo', 'reese', 'giant', 'pride', 'nou...","[('nough', 'NN'), ('said', 'VBD')]","['nough', 'say']","[[, ', n, o, u, g, h, ', ,, , ', s, a, y, ', ]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301205,301205,9.182763e+17,.@Chevron: We are deeply saddened by the loss ...,2017-10-12,NaN,en,True,4.0,1.0,7.0,...,NaN,NaN,NaN,NaN,NaN,deeply saddened loss life devastation caused w...,"['deeply', 'saddened', 'loss', 'life', 'devast...","[('deeply', 'RB'), ('saddened', 'VBN'), ('loss...","['deeply', 'sadden', 'loss', 'life', 'devastat...","[[, ', d, e, e, p, l, y, ', ,, , ', s, a, d, ..."
301206,301206,9.182728e+17,So sad. â¡ï¸ âCouple married for 75 years ...,2017-10-12,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,sad couple married 75 years die together calif...,"['sad', 'couple', 'married', 'years', 'die', '...","[('sad', 'JJ'), ('couple', 'NN'), ('married', ...","['sad', 'couple', 'marry', 'year', 'die', 'tog...","[[, ', s, a, d, ', ,, , ', c, o, u, p, l, e, ..."
301207,301207,9.182711e+17,3Novices:Death Toll Climbs to 21 From Wildfire...,2017-10-12,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,3novicesdeath toll climbs 21 wildfires norther...,"['novicesdeath', 'toll', 'climbs', 'wildfires'...","[('3novices', 'NNS'), ('death', 'NN'), ('toll'...","['3novices', 'death', 'toll', 'climb', 'wildfi...","[[, ', 3, n, o, v, i, c, e, s, ', ,, , ', d, ..."
301208,301208,9.196318e+17,So sad - cartoonist Brian Fies' comic about lo...,2017-10-15,NaN,en,False,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,sad cartoonist brian fies comic losing home ca...,"['sad', 'cartoonist', 'brian', 'fies', 'comic'...","[('sad', 'JJ'), ('cartoonist', 'NN'), ('brian'...","['sad', 'cartoonist', 'brian', 'fies', 'comic'...","[[, ', s, a, d, ', ,, , ', c, a, r, t, o, o, ..."


# Find dimension

In [ ]:
# Create an instance of CountVectorizer
vectorizer = CountVectorizer(
                             max_features = 1000)

# document_texts = [' '.join(tokens) for tokens in df['lemm_text']]
document_texts = [tokens for tokens in filtered_df['lemm_text']]


# Fit the vectorizer on the 'lemm_text' column
vectorizer.fit(document_texts)

# Transform the 'lemm_text' column into a count vector representation
count_vector = vectorizer.transform(document_texts)

# Get the dimensions of the count vector
vector_dimensions = count_vector.shape

# Print the dimensions
print("Count Vector Dimensions:", vector_dimensions)


Count Vector Dimensions: (81423, 1000)


# SK LDA

In [ ]:
# #try from 10 to 100 topics iterations
# X = count_vector

# topic_range = range(10,101,10)
# perplexities = []

# for i in topic_range:
#   ldask = LatentDirichletAllocation(n_components = i, random_state = 5566)
#   ldask.fit(X)
#   perplexity = ldask.perplexity(X)
#   perplexities.append(perplexity)
#   print("Number of topics: {}, Perplexity: {}".format(i, perplexity))


In [ ]:
# # Print the top words for each topic

# ldask30 = LatentDirichletAllocation(n_components = 30, random_state = 5566)
# ldask30.fit(X)

# feature_names = vectorizer.get_feature_names_out()
# for topic_idx, topic in enumerate(ldask30.components_):
#     print(f"Topic #{topic_idx + 1}:")
#     top_words_idx = topic.argsort()[:-16:-1]  # Select top 15 words
#     top_words = [feature_names[i] for i in top_words_idx]
#     print(top_words)
#     print()

In [ ]:
# save model
# Save the LDA model
# with open('drive/MyDrive/5.TopicModeling/Model/lda_during_wildfire_sk', 'wb') as file:
#     pickle.dump(ldask30, file)

In [ ]:
# # Load the LDA model
# model_path = 'drive/MyDrive/5.TopicModeling/Model/lda_during_wildfire_sk'
# with open(model_path, 'rb') as file:
#     lda_dw_sk = pickle.load(file)

# for topic_idx, topic in enumerate(lda_dw_sk.components_):
#     print(f"Topic #{topic_idx + 1}:")
#     top_words_idx = topic.argsort()[:-16:-1]  # Select top 15 words
#     top_words = [feature_names[i] for i in top_words_idx]
#     print(top_words)
#     print()

In [ ]:
# # Define the objective function for optimization
# def objective(trial):
#     n_topics = trial.suggest_int('n_topics', 10, 100)
#     alpha = trial.suggest_uniform('alpha', 0.1, 1.0)
#     beta = trial.suggest_uniform('beta', 0.1, 1.0)

#     lda = LatentDirichletAllocation(
#         n_components=n_topics,
#         doc_topic_prior=alpha,
#         topic_word_prior=beta,
#         random_state=42
#     )

#     lda.fit(X)
#     perplexity = lda.perplexity(X)
#     return perplexity

# # Run the optimization
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=50)

# # Print the best hyperparameters and objective value
# best_params = study.best_params
# best_value = study.best_value
# print("Best Hyperparameters: ", best_params)
# print("Best Objective Value: ", best_value)

In [ ]:
# # Print the top words for each topic

# ldask_mod = LatentDirichletAllocation(n_components = 12, random_state = 5566,
#                                     doc_topic_prior = 0.10032827992479586, topic_word_prior = 0.10511489014645924)
# ldask_mod.fit(X)

# feature_names = vectorizer.get_feature_names_out()
# for topic_idx, topic in enumerate(ldask_mod.components_):
#     print(f"Topic #{topic_idx + 1}:")
#     top_words_idx = topic.argsort()[:-16:-1]  # Select top 15 words
#     top_words = [feature_names[i] for i in top_words_idx]
#     print(top_words)
#     print()

# Gensim LDA

In [ ]:
# Create a list of lists from the preprocessed documents
# processed_docs = [doc.split() for doc in df['lemm_text']]
processed_docs = [re.sub(r'[\[\],]', '', doc).split() for doc in filtered_df['lemm_text']]
dictionary = corpora.Dictionary(processed_docs)
# dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)  # Limit the dictionary size
# Convert the dictionary into a bag-of-words representation
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
# Initialize the LDA model with 10 top
num_topics = 10
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, random_state = 5566)

In [ ]:
# Increase the number of topics from 10 to 100
new_num_topics = 100

while num_topics < new_num_topics:
    # Increase the number of topics
    num_topics += 10

    # Re-train the LDA model with the updated number of topics
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary)

    # Evaluate the model's performance (using coherence score as an example)
    coherence_score = models.CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v').get_coherence()

    print(f"Number of topics: {num_topics}, Coherence Score: {coherence_score}")

    # Repeat the process until the desired number of topics or satisfactory results are achieved

Number of topics: 20, Coherence Score: 0.3257249003340329
Number of topics: 30, Coherence Score: 0.33566066046059245
Number of topics: 40, Coherence Score: 0.3625941371363871
Number of topics: 50, Coherence Score: 0.39226988064641616
Number of topics: 60, Coherence Score: 0.4330458655316968
Number of topics: 70, Coherence Score: 0.4297123292204261
Number of topics: 80, Coherence Score: 0.41786233854256005
Number of topics: 90, Coherence Score: 0.3812857901127664
Number of topics: 100, Coherence Score: 0.3563034649886238


In [ ]:
#try new topic model
lda_modelA = models.LdaModel(corpus, num_topics=60, id2word=dictionary)
for i,topic in lda_modelA.show_topics(formatted=True, num_topics=60, num_words=15):
    print(str(i)+": "+ topic)
    print()

#Parameter Modification (Optuna)

In [ ]:
def objective(trial):
    num_topics = trial.suggest_int('num_topics', 10, 100)
    # iterations = trial.suggest_int('iterations', 50, 100)
    alpha = trial.suggest_categorical('alpha', ['symmetric', 'asymmetric'])
    eta = trial.suggest_float('eta', 0.1, 1.0)
    # rounded_eta = round(suggested_eta, 2)

    # Handle None values for hyperparameters
    # alpha = 1.0 if alpha is None else alpha
    # beta = 1.0 if beta is None else beta

    # Train the LDA model with the current hyperparameters
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics,
                         alpha=alpha, eta=eta)

    # Calculate coherence score for the model
    coherence_model = models.CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    return coherence_score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Print the best hyperparameters and coherence score
best_params = study.best_params
best_coherence = study.best_value
print("Best number of topics:", best_params['num_topics'])
print("Best alpha:", best_params['alpha'])
print("Best beta:", best_params['eta'])
print("Best coherence score:", best_coherence)


[I 2023-06-29 04:58:57,061] A new study created in memory with name: no-name-e973a303-5215-40ee-8dff-8d1b0eb395fb
[I 2023-06-29 04:59:52,794] Trial 0 finished with value: 0.41172230954855166 and parameters: {'num_topics': 78, 'alpha': 'symmetric', 'eta': 0.7769296655706582}. Best is trial 0 with value: 0.41172230954855166.
[I 2023-06-29 05:00:36,615] Trial 1 finished with value: 0.40370986842432655 and parameters: {'num_topics': 69, 'alpha': 'symmetric', 'eta': 0.8535866220164786}. Best is trial 0 with value: 0.41172230954855166.
[I 2023-06-29 05:01:14,788] Trial 2 finished with value: 0.4492364898852266 and parameters: {'num_topics': 48, 'alpha': 'symmetric', 'eta': 0.6690895760570656}. Best is trial 2 with value: 0.4492364898852266.
[I 2023-06-29 05:02:02,646] Trial 3 finished with value: 0.49219966753514904 and parameters: {'num_topics': 77, 'alpha': 'asymmetric', 'eta': 0.22976721446366735}. Best is trial 3 with value: 0.49219966753514904.
[I 2023-06-29 05:02:33,351] Trial 4 finish

Best number of topics: 88
Best alpha: asymmetric
Best beta: 0.23525127417199476
Best coherence score: 0.506675084021305


In [ ]:
modified_lda_model = LdaModel(corpus, id2word = dictionary,
                       num_topics = best_params['num_topics'], alpha = best_params['alpha'],
                       eta = best_params['eta'])
for i,topic in modified_lda_model.show_topics(formatted=True, num_topics=best_params['num_topics'], num_words=15):
    print(str(i)+": "+ topic)
    print()

# Best number of topics: 88
# Best alpha: asymmetric
# Best beta: 0.23525127417199476
# Best coherence score: 0.506675084021305

In [ ]:
# save model
# with open('drive/MyDrive/5.TopicModeling/Model/lda_duringWildfire_mod', 'wb') as file:
#     pickle.dump(modified_lda_model, file)

In [13]:
# Load the LDA model
model_path = 'drive/MyDrive/5.TopicModeling/Model/lda_duringWildfire_mod'
with open(model_path, 'rb') as file:
    modified_lda_model = pickle.load(file)

topics = []
#implement model
for i,topic in modified_lda_model.show_topics(formatted=True, num_topics=88, num_words=15):
    topics.append(topic)
    print(str(i)+": "+ topic)
    print()

0: 0.128*"'fire'" + 0.116*"'angry'" + 0.090*"'say'" + 0.036*"'california'" + 0.034*"'smoke'" + 0.027*"'high'" + 0.021*"'wildfire'" + 0.020*"'long'" + 0.018*"'deadly'" + 0.017*"'close'" + 0.014*"'school'" + 0.014*"'pay'" + 0.011*"'pot'" + 0.010*"'chicago'" + 0.010*"'room'"

1: 0.427*"'smoke'" + 0.059*"'day'" + 0.045*"'weed'" + 0.024*"'cigarette'" + 0.016*"'bore'" + 0.015*"'depression'" + 0.014*"'hand'" + 0.013*"'fall'" + 0.013*"'second'" + 0.012*"'part'" + 0.010*"'washington'" + 0.007*"'wildfire'" + 0.007*"'mad'" + 0.007*"'fire'" + 0.006*"'ravage'"

2: 0.206*"'get'" + 0.130*"'fire'" + 0.060*"'know'" + 0.032*"'dumpster'" + 0.027*"'tired'" + 0.024*"'ever'" + 0.018*"'liar'" + 0.012*"'money'" + 0.012*"'hour'" + 0.009*"'sex'" + 0.008*"'burn'" + 0.008*"'map'" + 0.007*"'nervous'" + 0.007*"'chill'" + 0.007*"'less'"

3: 0.285*"'sad'" + 0.181*"'fire'" + 0.092*"'tire'" + 0.032*"'still'" + 0.029*"'good'" + 0.026*"'work'" + 0.024*"'hear'" + 0.015*"'amp'" + 0.012*"'california'" + 0.011*"'turn'" + 0.0

In [15]:
# Calculate and summarize the total percentage for each topic
for i, topic in enumerate(topics):
    words = topic.split(" + ")
    total_percentage = sum(float(w.split("*")[0]) for w in words)
    print(f"Topic {i}: Total Percentage = {total_percentage:.2%}")

Topic 0: Total Percentage = 56.60%
Topic 1: Total Percentage = 67.50%
Topic 2: Total Percentage = 56.70%
Topic 3: Total Percentage = 75.10%
Topic 4: Total Percentage = 60.40%
Topic 5: Total Percentage = 45.00%
Topic 6: Total Percentage = 48.00%
Topic 7: Total Percentage = 34.70%
Topic 8: Total Percentage = 32.90%
Topic 9: Total Percentage = 46.50%
Topic 10: Total Percentage = 48.50%
Topic 11: Total Percentage = 30.20%
Topic 12: Total Percentage = 58.10%
Topic 13: Total Percentage = 55.80%
Topic 14: Total Percentage = 56.00%
Topic 15: Total Percentage = 53.90%
Topic 16: Total Percentage = 0.00%
Topic 17: Total Percentage = 32.30%
Topic 18: Total Percentage = 45.00%
Topic 19: Total Percentage = 49.70%
Topic 20: Total Percentage = 44.80%
Topic 21: Total Percentage = 48.90%
Topic 22: Total Percentage = 45.00%
Topic 23: Total Percentage = 22.20%
Topic 24: Total Percentage = 51.80%
Topic 25: Total Percentage = 23.80%
Topic 26: Total Percentage = 24.40%
Topic 27: Total Percentage = 46.00%
Top

# Topic Definition

### Topic 0: Fire-related Anger and California Wildfires

The prominent keywords such as "fire," "angry," and "say" indicate a focus on anger or frustration related to fires. The mention of "California" and "smoke" suggests a connection to wildfires in that region. Other keywords like "high," "wildfire," and "deadly" further reinforce the fire-related context. Additional keywords like "long," "close," and "school" may suggest specific situations or locations associated with the topic. However, without further context, it's challenging to determine their exact significance.

### Topic 1: Smoking and its Effects

Based on the provided keywords, the prominent theme of this topic appears to be related to smoking, specifically "smoke," "weed," and "cigarette." Additional keywords such as "day," "bore," "depression," and "fall" might suggest a connection to personal experiences or emotions associated with smoking.

### Topic 20: Anxiety and Television Show/Fandom

The most prominent keyword is "anxiety," indicating a focus on anxiety-related discussions. Additionally, words like "show" and "fan" suggest a connection to television shows and fandoms. The presence of terms like "insomnia" and "wildfire" might imply specific contexts or themes within the broader topic. However, without further information or context, it is difficult to determine the exact nature of these connections. Therefore, the topic label "Anxiety and Television Show/Fandom" captures the main themes represented by the keywords provided.

### Topic 31: Air Quality and Concerns

The keywords like "'air'", "'problem'", "'quality'", "'isolation'", and "'poor'" suggest a discussion or mention of issues related to air quality. Additionally, terms like "'suffocate'", "'cracker'", and "'zombie'" imply potential dangers or negative effects associated with the topic. The inclusion of words such as "'session'", "'check'", and "'clearly'" suggests some sort of evaluation or examination of the air quality situation. Overall, these keywords indicate a focus on the quality of air and potential problems or concerns related to it.

### Topic 44: Emotional Exhaustion and Hope

The prominent keywords such as "'man'", "'tired'", "'everyone'", and "'hope'" indicate a sense of weariness and emotional fatigue. Additionally, words like "'service'", "'tense'", and "'firework'" suggest a potential connection to events or situations that may contribute to this exhaustion. The presence of "'hope'" indicates a glimmer of optimism or a desire for positive change. Considering these elements, the topic can be described as "Emotional Exhaustion and Hope."


### Topic 56: Damage Control and Acts of Saving

The prominent keywords in the list include "damage," "act," and "save," which indicate a focus on mitigating or addressing damage and taking action to save or protect something or someone. Other related keywords like "London," "racist," "archive," and "bird" suggest specific contexts or aspects within this broader theme.

### Topic 73: Watching Flames and Depression

The most prominent keyword is "'watch'," indicating a focus on observing or monitoring something. The presence of "'flame'" suggests a specific object of observation related to fire or light. Additionally, the keyword "'depression'" indicates that the topic involves discussions or concerns related to mental health. The other keywords, such as "'fully'", "'race'", "'friendly'", "'condition'", "'private'", "'binge'", "'period'", "'detail'", "'Netflix'", "'haunt'", "'India'", and "'Miami'", provide additional context but are not as prominent in the topic.

### Topic 76: Strong Negative Emotions and Disturbing Experiences

he presence of words like "'fuck'," "'scary'," "'hate'," and "'shock'" indicates strong negative emotions. Additionally, words like "'fire'," "'gun'," and "'california'" suggest potentially distressing or alarming experiences. The keywords "'book'" and "'write'" might indicate a connection to literature or writing, but given their relatively low weights in the list, they don't appear to be central to the topic. Overall, this topic seems to revolve around expressing intense negative emotions and discussing unsettling experiences.

### Topic 81: Trump

The most prominent keyword in the list is "'Trump," which indicates a focus on or mention of former U.S. President Donald Trump. The word "'match" suggests a potential discussion or context involving matches, which could refer to sports matches, matches as in lighting a fire, or even metaphorical matches. Other keywords like "'folk," "'spain," "'shin," and "'kitchen" provide additional context but do not significantly alter the primary focus on Trump and matches.